In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# 
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
# 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[72,  1, 54, 67, 57,  1, 59, 62],
        [58, 57,  1, 54,  1, 73, 61, 62],
        [59, 62, 71, 72, 73,  1, 73, 61],
        [68, 71, 71, 78,  9,  3,  1, 28]])
targets:
tensor([[ 1, 54, 67, 57,  1, 59, 62, 72],
        [57,  1, 54,  1, 73, 61, 62, 71],
        [62, 71, 72, 73,  1, 73, 61, 62],
        [71, 71, 78,  9,  3,  1, 28, 68]])


In [7]:
tensor1=torch.tensor([1.0,2.0,3.0])
softmax_output=F.softmax(tensor1,dim=0)
print(softmax_output)

tensor([0.0900, 0.2447, 0.6652])


In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


;m(Lvvs﻿c5Y'tx6YmMi9_Q:f-m﻿pGA79bxq7s&ALY'I5Zb*Y67!Kc0,[j:Cih﻿tm'osyFYHsMwGj'6-N"c'US[ql1f-4j'u﻿emyWhmmkqBFDg0FeAfIavZg3o:50x6-;k*tZm"6OBO3﻿M!f.DoWT-IP[JailD﻿FGqy!0L
KvE1E4H&b_k H5u:S_5ZQbi8P;5,c5Xqs0DAw9]P&,Fi Ox_"**t
LtDXMwhE8v-AyTYdFDB.Y4,7J*ldv,F .pYS39Dj6Nhpb_dEq4DMP)1cY47Vr-]z5ZUsLz'OORb_kbHiXp68bEgxLqlt_oOJaaVLUTCxv:T8F.g7))yUb9dV,[&1s&vUkQ0xTSXxG1c? 5k98j.hUSWq07tsr 58IRswOH]bk]vHkdwr0bWoZ&uj)2l69LZOJYEp!:'[oI2PT0X9d37MJdCxvsOBW)[!C0!B1eyiDmElJi 5333dFwiNLV?SdHPZ'[&BqjG7)19ThqN)m﻿9OKiC5d


In [12]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.851, val loss: 4.847
step: 250, train loss: 4.787, val loss: 4.787
step: 500, train loss: 4.743, val loss: 4.722
step: 750, train loss: 4.668, val loss: 4.650
4.370579242706299


In [19]:
# lookup tril 对角线
input_matrix=torch.ones(5,5)
print(input_matrix)
output_matrix=torch.tril(input_matrix)
print(output_matrix)
output_matrix=torch.triu(input_matrix)
print(output_matrix)
# 
input_matrix_0=torch.zeros(5,5)
output=input_matrix_0.masked_fill(output_matrix==0,float('-inf'))
print(output)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])
tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]])
tensor([[0., 0., 0., 0., 0.],
        [-inf, 0., 0., 0., 0.],
        [-inf, -inf, 0., 0., 0.],
        [-inf, -inf, -inf, 0., 0.],
        [-inf, -inf, -inf, -inf, 0.]])
